# Prediction with ForceRegression model

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_kedro
import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from src.vct_scaling import scale_force_to_model_scale, scale_moment_to_model_scale


In [ ]:
id = "22773"
df = catalog.load(f"{id}.data")
df_smooth = catalog.load(f"{id}.data_ek_smooth")
model = catalog.load("vmm_martins_simple.force_regression.model")
regression = catalog.load("vmm_martins_simple.force_regression.regression")
df_VCT = catalog.load("force_regression.data_scaled")
ship_data = catalog.load("ship_data")

model_motion_regression = catalog.load("vmm_martins_simple.motion_regression.joined.model")

#keys_forces = ['fx','fy','fz']
#scale_factor=ship_data['scale_factor']
#forces=df_VCT[keys_forces]
#df_VCT[keys_forces] = scale_force_to_model_scale(forces=forces, scale_factor=scale_factor, **df_VCT)
#keys_moments = ['mx','my','mz']
#df_VCT[keys_moments] = scale_force_to_model_scale(df_VCT[keys_moments], scale_factor=scale_factor, **df_VCT)


In [ ]:
df_VCT[['u','v','V','r','fx','fy','mz']].abs().max()

In [ ]:
model.parameters

In [ ]:
model.parameters['Xu'] = -0.000737*1.7
result = model.simulate(df_=df_smooth)

In [ ]:
result.result.plot(y='U')

In [ ]:
dataframes = {
'model test' : df_smooth,
'simulation (model test)' : result.result,    
}

fig,ax=plt.subplots()
track_plots(dataframes=dataframes, lpp=model.ship_parameters['L'], beam=model.ship_parameters['B'], ax=ax, plot_boats=False);

plot(dataframes, keys=result.result.columns);

In [ ]:
data = df_VCT[['fx','fy','mz','u','v','V','r','beta','delta','thrust','test type']].copy()
df_model = data.copy()
df_model[['fx','fy','mz']] = model.forces(inputs=df_model)
df_model['item'] = 'regression'

In [ ]:
df = data.copy()
df['item'] = 'VCT'
df = df.append(df_model)

captive_plot(df, styles=['.','-'])